Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import operator
from scipy.stats import randint, uniform
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_validate

Import Models

In [2]:
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# NOT RECOMMENDED! ignore warnings
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

***

In [4]:
# TODO: Correlation summary

In [4]:
class SingleModelSearch:
    """
    TODO
        1. Review hyperparameters grid. 
        3. Add average training AUC
    
    Search for the best model type using cross validation.
    
    Parameters
    ----------
    n_iter: int
        Number of parameter settings that are sampled.
    cv: int
        Number of cross validation folds.
    scoring: string, default='roc_auc'
        Scoring metrics (e.g: AUC, Accuracy, F1, etc)
    n_jobs: int, default=None
        Number of thread to use for parallel computing.
    random_state: int, default=None
        Random seed for RandomizedSearchCV().
        Set seed to integer to get reproducible result.
    verbose: boolean, default=False
        Set to True to print messages when searching in progress.
        
    Attributes
    ----------
    model_search_params_: dict
        Contains all the optimal CV parameters for every searched model.
    model_search_score_: DataFrame
        Contains all the best CV score for every searched model.
    best_model_: dict
        Contains the name, parameters and score of best model.
    """
    _model = {
        'logistic': LogisticRegression(max_iter=1000),
        'lda': LinearDiscriminantAnalysis(), 
        'naivebayes': GaussianNB(),
        'rf': RandomForestClassifier(n_estimators=100),
        'xgboost': XGBClassifier(),
        'knn': KNeighborsClassifier(),
        'svc': SVC(kernel='rbf'),
        'mlp': MLPClassifier(max_iter=500)
    }
    
    _params_grid = {
        'logistic': {
            'solver' : ['liblinear', 'saga'],
            'C' : [1e-3, 1e-2, 0.1, 1, 10, 100]
        },
        'rf': {
            'max_depth': randint(10, 100),
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': randint(1, 4),
            'min_samples_split': randint(2, 10),
            'bootstrap': [True, False]
        },
        'xgboost': {
            'max_depth': randint(1,6),
            'min_child_weight': randint(0,6),
            'subsample': uniform(loc=0.6, scale=0.4),
            'colsample_bytree': uniform(loc=0.6, scale=0.4),
            'gamma': [i/10.0 for i in range(0,5)],
            'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100]
        }, 
        'knn': {
            'n_neighbors': randint(1, 100)
        },
        'svc': {
            'gamma': [0.1, 1, 10, 100],
            'C': [0.1, 1, 10, 100, 1000]
        },
        'mlp': {
            'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
            'activation': ['tanh', 'relu'],
            'solver': ['sgd', 'adam'],
            'alpha': [0.0001, 0.05],
            'learning_rate': ['constant','adaptive']
        }
    }
    def __init__(self, n_iter, cv, scoring='roc_auc', n_jobs=None,  
                 random_state=None, verbose=False):
        self.n_iter = n_iter
        self.scoring = scoring
        self.n_jobs = n_jobs
        self.cv = cv
        self.random_state = random_state
        self.verbose = verbose
        
    def _get_params_score(self, X, y, model):
        """
        Find the optimal CV parameters and score for each model types.
        
        Parameters
        ----------
        X: DataFrame
            Input features
        Y: DataFrame
            Input label
        model: string
            Name of the model to be tested. (See class variables)
        
        Returns
        -------
        params: dict
            Best CV model parameters.
        score: int
            Best CV score. 
        """
        
        # valid model name that has searching grid
        if model in self._params_grid.keys():
            rand_search = RandomizedSearchCV(
                estimator = self._model[model],
                param_distributions = self._params_grid[model],
                scoring = self.scoring,
                n_iter = self.n_iter,
                cv = self.cv,
                n_jobs = self.n_jobs,
                random_state = self.random_state)
            
            rand_search.fit(X, y)
            
            return rand_search.best_params_, rand_search.best_score_
        # valid model name without searching grid
        elif model in self._model.keys():
            cv_fit = cross_validate(self._model[model], X, y, 
                                    scoring=self.scoring, cv=self.cv, 
                                    n_jobs=self.n_jobs)
            
            return self._model[model].get_params(), cv_fit['test_score'].mean()
        else:
            raise Exception('Invalid model! Please input a valid model name')
    
    
    def fit(self, X, y):
        """
        Parameters
        ----------
        X: DataFrame
            Input features
        Y: DataFrame
            Input label
        """
        # declare dtype for desired output
        self.model_search_params_ = {}
        self.model_search_score_ = pd.DataFrame([], columns=['model', 'score'])
        self.best_model_ = {}
        
        for model in self._model.keys():
            if self.verbose == True:
                print("Currently testing for {}".format(model))
                
            # find the CV results and optimal params
            params, score= self._get_params_score(X, y, model)
            
            # record best CV params & score
            self.model_search_params_[model] = params
            self.model_search_score_ = self.model_search_score_.\
                                       append({'model': model, 'score': score}, 
                                              ignore_index=True)
        
        # sort search results in descending order
        self.model_search_score_ = self.model_search_score_.\
                                   sort_values(by='score', ascending=False).\
                                   reset_index(drop=True)
        
        # extract the name, params and score for best model
        best_model = self.model_search_score_.\
                     loc[self.model_search_score_.score.idxmax(), :]
        
        self.best_model_['name'] = best_model.model
        self.best_model_['score'] = best_model.score
        self.best_model_['params'] = self.model_search_params_[best_model.model]
        

In [5]:
class BlendedModelSearch(SingleModelSearch):
    """
    TODO:
        1. Suggest candidate models (Check correlation - choose uncorrelated)
        2. Add average training AUC
    
    Search for the candidate models for blending using cross validation.
    
    Parameters
    ----------
    frac: float, range within (0, 1]
        Proportion of features that are sampled.
    n_experiments:
        Number of times to repeat the sampling plus searching process.
    n_iter: int
        Number of parameter settings that are sampled.
    cv: int
        Number of cross validation folds.
    scoring: string, default='roc_auc'
        Scoring metrics (e.g: AUC, Accuracy, F1, etc)
    n_jobs: int, default=None
        Number of thread to use for parallel computing.
    random_state: int, default=None
        Random seed for RandomizedSearchCV() and features sampling.
        Set seed to integer to get reproducible result.
    verbose: boolean, default=False
        Set to True to print messages when searching in progress.
        
    Attributes
    ----------
    model_search_params_: dict
        Contains all the optimal CV parameters for every searched model.
    model_search_score_: DataFrame
        Contains all the best CV score for every searched model.
    
    """
    def __init__(self, frac, n_experiment, n_iter, cv, scoring='roc_auc', 
                 n_jobs=None, random_state=None, verbose=False):
        super().__init__(n_iter=n_iter, cv=cv, scoring=scoring, n_jobs=n_jobs,
                         random_state=random_state, verbose=verbose)
        self.frac = frac
        self.n_experiment = n_experiment
        
    def fit(self, X, y):
        """
        Parameters
        ----------
        X: DataFrame
            Input features
        Y: DataFrame
            Input label
        """
        
        # declare dtype for the desired output
        self.model_search_params_ = {}
        self.model_search_score_ = pd.DataFrame(
            [], 
            columns=['experiment', 'model', 'score'])
        
        for i in range(self.n_experiment):
            
            # sample the features randomly
            X_sample = X.sample(frac=self.frac, axis=1,
                               random_state=self.random_state + i)
            y_sample = y
            
            # test for all model types
            for model in self._model.keys():
                if self.verbose == True:
                    print("Experiment {}: Testing for {}".format(i, model))
                
                # find the CV results and optimal params
                params, score = super()._get_params_score(
                    X_sample, y_sample, model)
                
                # save the searched parameters
                self.model_search_params_[(i, model)] = \
                    {'params': params, 'feature': list(X_sample.columns)}
                
                # save the search results
                self.model_search_score_ = \
                    self.model_search_score_.append(
                        {'experiment': i, 'model': model, 'score': score},
                         ignore_index=True)
        
        # sort the search results in descending order
        self.model_search_score_ = self.model_search_score_.\
                                   sort_values(by='score', ascending=False)\
                                   .reset_index(drop=True)

Software Testing

In [6]:
# data = pd.read_csv('feat.csv', encoding = "ISO-8859-1", low_memory=False)
# data = data.loc[:, ['AGE', 'DEPENDENT_CNT', 'DELQ_FLG']].dropna().reset_index(drop=True)
# data['DELQ_FLG'] = data['DELQ_FLG'].map({'N': 0, 'Y': 1})

In [7]:
data = pd.read_csv("data.csv")

In [8]:
# Test single model mode
msearch = SingleModelSearch(n_iter=1, cv=3, verbose=True, n_jobs=-1)
msearch.fit(data.loc[:, ~data.columns.isin(['bookingID', 'label'])], data['label'])
msearch.model_search_score_

Currently testing for mlp
Currently testing for naivebayes
Currently testing for lda
Currently testing for svc
Currently testing for logistic
Currently testing for knn
Currently testing for rf
Currently testing for xgboost


,model,score
0,lda,0.715709
1,rf,0.714227
2,xgboost,0.711938
3,naivebayes,0.695930
4,knn,0.688711
5,logistic,0.684187
6,mlp,0.608873
7,svc,0.500000


In [13]:
# Test blended mode
msearch = BlendedModelSearch(frac=0.5, n_experiment=2, n_iter=5, 
                             cv=3, random_state=89,
                             n_jobs=-1,verbose=True)
msearch.fit(data.loc[:, ~data.columns.isin(['bookingID', 'label'])], data['label'])
msearch.model_search_score_

Experiment 0: Testing for mlp
Experiment 0: Testing for naivebayes
Experiment 0: Testing for lda
Experiment 0: Testing for svc
Experiment 0: Testing for logistic
Experiment 0: Testing for knn
Experiment 0: Testing for rf
Experiment 0: Testing for xgboost
Experiment 1: Testing for mlp
Experiment 1: Testing for naivebayes
Experiment 1: Testing for lda
Experiment 1: Testing for svc
Experiment 1: Testing for logistic
Experiment 1: Testing for knn
Experiment 1: Testing for rf
Experiment 1: Testing for xgboost


,experiment,model,score
0,0,logistic,0.732086
1,0,lda,0.731074
2,1,lda,0.729299
3,1,xgboost,0.728737
4,1,logistic,0.726572
5,0,xgboost,0.723179
6,1,rf,0.718343
7,0,rf,0.699245
8,0,knn,0.691859
9,1,naivebayes,0.684351
